# L3: Supervised Fine-Tuning (SFT) 監督式微調

## 課程概述

在這個課程中，我們將學習監督式微調（SFT）的基本概念和實作方法。SFT是一種將基礎語言模型轉換為能夠遵循指令的對話模型的重要技術。

### 主要學習目標：
1. **理解 SFT 的基本原理**：學習如何透過模仿範例回應來訓練模型
2. **掌握 SFT 的工作流程**：從資料準備到模型訓練的完整過程
3. **實作 SFT 訓練**：使用真實資料集進行模型微調
4. **比較訓練前後的效果**：觀察模型在微調前後的差異

### 課程重點：
- **SFT 的數學原理**：負對數似然損失函數的最小化
- **資料品質的重要性**：高品質資料比大量資料更重要
- **參數效率微調**：LoRA 等技術的應用
- **實際應用案例**：從基礎模型到指令模型的轉換

In [2]:
!git clone https://github.com/sheep52031/DeepLearning.AI_SelfStudy-Notes.git
!ls
%cd DeepLearning.AI_SelfStudy-Notes

Cloning into 'DeepLearning.AI_SelfStudy-Notes'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 23 (delta 3), reused 18 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (23/23), 10.71 KiB | 10.71 MiB/s, done.
Resolving deltas: 100% (3/3), done.
DeepLearning.AI_SelfStudy-Notes  Post-training_of_LLMs	README.md
/kaggle/working/DeepLearning.AI_SelfStudy-Notes/DeepLearning.AI_SelfStudy-Notes


In [ ]:
import os
base_dir = 'DeepLearning.AI_SelfStudy-Notes/Post-training_of_LLMs'


In [ ]:
!pip install -r Post-training_of_LLMs/requirements.txt --no-deps

## 匯入必要的函式庫

這個部分我們將匯入進行 SFT 訓練所需的核心函式庫：

In [ ]:
import torch  # PyTorch 深度學習框架
import pandas as pd  # 資料處理和分析
from datasets import load_dataset, Dataset  # HuggingFace 資料集載入
from transformers import TrainingArguments, AutoTokenizer, AutoModelForCausalLM  # 模型和分詞器
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM, SFTConfig  # SFT 訓練工具

## 設定輔助函式

這些輔助函式將幫助我們進行模型載入、回應生成和測試等操作。

In [ ]:
def generate_responses(model, tokenizer, user_message, system_message=None, 
                       max_new_tokens=100):
    """
    生成模型回應的函式
    
    參數:
    - model: 語言模型
    - tokenizer: 分詞器
    - user_message: 使用者訊息
    - system_message: 系統訊息（可選）
    - max_new_tokens: 生成的最大新詞元數量
    
    返回:
    - response: 模型生成的回應
    """
    # 使用分詞器的聊天模板格式化對話
    messages = []
    if system_message:
        messages.append({"role": "system", "content": system_message})
    
    # 我們假設資料都是單輪對話
    messages.append({"role": "user", "content": user_message})
        
    # 應用聊天模板
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False,
    )

    # 將提示轉換為模型輸入格式
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    # 生成回應（建議使用 vllm、sglang 或 TensorRT 以獲得更好的效能）
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,  # 使用貪心搜索
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    
    # 提取生成的部分
    input_len = inputs["input_ids"].shape[1]
    generated_ids = outputs[0][input_len:]
    response = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()

    return response

In [ ]:
def test_model_with_questions(model, tokenizer, questions, 
                              system_message=None, title="Model Output"):
    """
    測試模型對一系列問題的回應
    
    參數:
    - model: 語言模型
    - tokenizer: 分詞器
    - questions: 問題列表
    - system_message: 系統訊息（可選）
    - title: 輸出標題
    """
    print(f"\n=== {title} ===")
    for i, question in enumerate(questions, 1):
        response = generate_responses(model, tokenizer, question, 
                                      system_message)
        print(f"\nModel Input {i}:\n{question}\nModel Output {i}:\n{response}\n")

In [ ]:
def load_model_and_tokenizer(model_name, use_gpu = False):
    """
    載入模型和分詞器
    
    參數:
    - model_name: 模型名稱或路徑
    - use_gpu: 是否使用 GPU
    
    返回:
    - model: 載入的模型
    - tokenizer: 載入的分詞器
    """
    # 載入基礎模型和分詞器
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    
    # 如果使用 GPU，將模型移動到 CUDA 設備
    if use_gpu:
        model.to("cuda")
    
    # 如果分詞器沒有聊天模板，則創建一個基本的模板
    if not tokenizer.chat_template:
        tokenizer.chat_template = """{% for message in messages %}
                {% if message['role'] == 'system' %}System: {{ message['content'] }}\n
                {% elif message['role'] == 'user' %}User: {{ message['content'] }}\n
                {% elif message['role'] == 'assistant' %}Assistant: {{ message['content'] }} <|endoftext|>
                {% endif %}
                {% endfor %}"""
    
    # 分詞器配置
    if not tokenizer.pad_token:
        tokenizer.pad_token = tokenizer.eos_token
        
    return model, tokenizer

In [ ]:
def display_dataset(dataset):
    """
    顯示資料集的前幾個範例
    
    參數:
    - dataset: 要顯示的資料集
    """
    # 視覺化資料集
    rows = []
    for i in range(3):
        example = dataset[i]
        # 提取使用者訊息
        user_msg = next(m['content'] for m in example['messages']
                        if m['role'] == 'user')
        # 提取助手回應
        assistant_msg = next(m['content'] for m in example['messages']
                             if m['role'] == 'assistant')
        rows.append({
            'User Prompt': user_msg,
            'Assistant Response': assistant_msg
        })
    
    # 顯示為表格
    df = pd.DataFrame(rows)
    pd.set_option('display.max_colwidth', None)  # 避免截斷長字串
    display(df)

## 載入基礎模型並測試簡單問題

首先我們載入一個基礎模型（未經過指令微調的模型），並測試它對簡單問題的回應能力。這將幫助我們理解 SFT 前後的差異。

In [ ]:
# 設定是否使用 GPU（在 Kaggle 環境中設為 True）
USE_GPU = True

# 定義測試問題
questions = [
    "Give me an 1-sentence introduction of LLM.",  # 要求簡短介紹 LLM
    "Calculate 1+1-1",  # 簡單數學計算
    "What's the difference between thread and process?"  # 技術概念解釋
]

In [30]:
from huggingface_hub import snapshot_download

snapshot_download(
    repo_id="Qwen/Qwen3-0.6B-Base",
    local_dir="./Qwen3-0.6B-Base",
    local_dir_use_symlinks=False
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:980: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

.gitattributes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

'/kaggle/working/DeepLearning.AI_SelfStudy-Notes/DeepLearning.AI_SelfStudy-Notes/Qwen3-0.6B-Base'

In [42]:
from transformers import AutoTokenizer, AutoModelForCausalLM

local_dir = "Qwen3-0.6B-Base"
tokenizer = AutoTokenizer.from_pretrained(local_dir, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(local_dir, trust_remote_code=True)

In [43]:

test_model_with_questions(model, tokenizer, questions, 
                          title="Base Model (Before SFT) Output")

del model, tokenizer


=== Base Model (Before SFT) Output ===

Model Input 1:
Give me an 1-sentence introduction of LLM.
Model Output 1:
⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ �


Model Input 2:
Calculate 1+1-1
Model Output 2:
⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ �


Model Input 3:
What's the difference between thread and process?
Model Output 3:
⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ �



## Qwen3-0.6B 模型的 SFT 結果

在這個部分，我們將檢視先前完成的 SFT 訓練結果。由於資源限制，我們不會在像 Qwen3-0.6B 這樣相對較大的模型上進行完整訓練。

### 對比分析：
- **基礎模型（SFT前）**：只會生成隨機符號，無法理解指令
- **微調模型（SFT後）**：能夠理解並回應使用者的問題

這個對比清楚地展示了 SFT 的威力 - 它能將一個只會預測下一個詞的基礎模型，轉換為能夠進行有意義對話的助手模型。

In [46]:
from huggingface_hub import snapshot_download

snapshot_download(
    repo_id="banghua/Qwen3-0.6B-SFT",
    local_dir="./Qwen3-0.6B-SFT",
    local_dir_use_symlinks=False
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:980: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

tokenizer.json: 0.00B [00:00, ?B/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

...

README.md: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/2.38G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

'/kaggle/working/DeepLearning.AI_SelfStudy-Notes/Qwen3-0.6B-SFT'

In [47]:
model, tokenizer = load_model_and_tokenizer("./Qwen3-0.6B-SFT", USE_GPU)

test_model_with_questions(model, tokenizer, questions, 
                          title="Base Model (After SFT) Output")

del model, tokenizer


=== Base Model (After SFT) Output ===

Model Input 1:
Give me an 1-sentence introduction of LLM.
Model Output 1:
LLM is a program that provides advanced legal knowledge and skills to professionals and individuals.


Model Input 2:
Calculate 1+1-1
Model Output 2:
1+1-1 = 2-1 = 1

So, the final answer is 1.


Model Input 3:
What's the difference between thread and process?
Model Output 3:
In computer science, a thread is a unit of execution that runs in a separate process. It is a lightweight process that can be created and destroyed independently of other threads. Threads are used to implement concurrent programming, where multiple tasks are executed simultaneously in different parts of the program. Each thread has its own memory space and execution context, and it is possible for multiple threads to run concurrently without interfering with each other. Threads are also known as lightweight processes.



## 在小型模型上進行 SFT 訓練

接下來我們將實際進行 SFT 訓練的完整流程。我們將使用一個較小的模型和資料集來確保訓練過程能在有限的計算資源上執行。

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>注意：</b> 我們在小型模型 <code>HuggingFaceTB/SmolLM2-135M</code> 和較小的訓練資料集上進行 SFT，以確保完整的訓練過程能在有限的計算資源上運行。如果你在自己的機器上運行筆記本並且有 GPU 資源，可以切換到更大的模型（如 <code>Qwen/Qwen3-0.6B-Base</code>）來進行完整的 SFT 訓練並重現上述結果。</p>
</div>

In [ ]:
# 選擇模型：在有 GPU 資源的情況下使用 Qwen3-0.6B-Base
# 如果資源有限，可以改用 HuggingFaceTB/SmolLM2-135M
#model_name = "./models/HuggingFaceTB/SmolLM2-135M"
model_name = "Qwen/Qwen3-0.6B-Base"
model, tokenizer = load_model_and_tokenizer(model_name, USE_GPU)

In [ ]:
# 載入訓練資料集
train_dataset = load_dataset("banghua/DL-SFT-Dataset")["train"]

# 如果沒有使用 GPU，則只使用前 100 個樣本進行訓練
if not USE_GPU:
    train_dataset = train_dataset.select(range(100))

# 顯示資料集的前幾個範例
print("訓練資料集範例：")
display_dataset(train_dataset)

print(f"\n資料集大小：{len(train_dataset)} 個樣本")
print("\n資料集特點：")
print("- 包含多樣化的指令和回應對")
print("- 涵蓋問答、翻譯、計算等多種任務")
print("- 每個樣本都包含使用者提示和助手回應")

In [ ]:
# SFT 訓練器配置
sft_config = SFTConfig(
    learning_rate=8e-5,  # 學習率：控制模型參數更新的步長
    num_train_epochs=1,  # 訓練輪數：設定模型訓練的完整迭代次數
    per_device_train_batch_size=1,  # 每設備批次大小：每個設備（如 GPU）訓練時的批次大小
    gradient_accumulation_steps=8,  # 梯度累積步驟：在執行反向傳播之前累積梯度的步驟數
    gradient_checkpointing=False,  # 梯度檢查點：以較慢的訓練速度為代價減少記憶體使用
    logging_steps=2,  # 記錄步驟：記錄訓練進度的頻率（每 2 步記錄一次）
)

# 關鍵超參數解釋：
print("SFT 訓練配置說明：")
print(f"學習率: {sft_config.learning_rate} - 決定模型參數更新的幅度")
print(f"訓練輪數: {sft_config.num_train_epochs} - 完整遍歷資料集的次數")
print(f"批次大小: {sft_config.per_device_train_batch_size} - 每次更新使用的樣本數")
print(f"梯度累積: {sft_config.gradient_accumulation_steps} - 有效批次大小 = 批次大小 × 梯度累積步驟")
print(f"有效批次大小: {sft_config.per_device_train_batch_size * sft_config.gradient_accumulation_steps}")

In [ ]:
# 建立 SFT 訓練器
sft_trainer = SFTTrainer(
    model=model,  # 要訓練的模型
    args=sft_config,  # 訓練配置
    train_dataset=train_dataset,  # 訓練資料集
    processing_class=tokenizer,  # 分詞器
)

print("開始 SFT 訓練...")
print("\nSFT 訓練過程說明：")
print("1. 模型將學習模仿訓練資料中的回應")
print("2. 透過最小化負對數似然損失來訓練")
print("3. 進度條將顯示訓練步驟和損失值")
print("4. 訓練完成後，模型將能夠回應使用者指令")

# 執行訓練
sft_trainer.train()

## 測試小型模型和小資料集的訓練結果

**注意：** 以下結果是針對我們在 SFT 訓練中使用的小型模型和資料集，這是由於計算資源有限。若要查看大型模型的完整訓練結果，請參閱上方的 **「Qwen3-0.6B 模型的 SFT 結果」** 部分。

### 預期結果分析：
由於我們使用的是小型模型（相對較少的參數）和有限的訓練資料，模型的表現可能會有以下特點：
- **有一定的改善**：相比未訓練的模型，應該能看到明顯的改善
- **仍有限制**：由於模型規模和資料量的限制，可能無法達到完美的表現
- **學習能力展現**：可以觀察到模型開始學會回應指令的基本能力

In [ ]:
# 如果沒有使用 GPU，將模型移到 CPU
if not USE_GPU:
    sft_trainer.model.to("cpu")

print("\n=== SFT 訓練完成！===\n")
print("現在來測試訓練後的模型表現...")

# 測試訓練後的模型
test_model_with_questions(sft_trainer.model, tokenizer, questions, 
                          title="訓練後的模型輸出")

print("\n=== 訓練效果分析 ===\n")
print("SFT 訓練的核心價值：")
print("1. 將基礎模型轉換為指令跟隨模型")
print("2. 透過模仿範例回應來學習對話模式")
print("3. 使模型能夠理解並回應使用者的各種請求")
print("\n即使在有限的資源下，我們也能觀察到模型在指令跟隨方面的改善！")

## 課程總結與深入思考

### 我們在這個課程中學到了什麼：

#### 1. SFT 的核心概念
- **定義**：監督式微調（SFT）是一種將基礎語言模型轉換為能夠遵循指令的對話模型的技術
- **原理**：通過最小化負對數似然損失函數，讓模型學習模仿訓練資料中的回應
- **目標**：使模型能夠理解並適當回應使用者的各種指令和問題

#### 2. SFT 的數學基礎
- **損失函數**：`Loss = -log P(response | prompt)`
- **訓練目標**：最大化給定提示下生成正確回應的概率
- **實現方式**：對所有訓練樣本的損失求和並進行梯度下降

#### 3. 資料品質的重要性
- **品質勝於數量**：1000個高品質樣本往往比100萬個混合品質的樣本效果更好
- **資料整理方法**：
  - 蒸餾：使用較大模型生成高品質回應
  - 最佳 k 選擇：從多個生成結果中選擇最佳回應
  - 篩選：根據品質和多樣性篩選大規模資料集

#### 4. 實際應用場景
- **模型行為啟動**：將預訓練模型轉為指令模型
- **能力改善**：提升特定任務的表現
- **知識蒸餾**：將大模型的能力轉移到小模型

### 運算環境選擇建議

關於您提到的運算環境選擇問題，讓我為您分析：

#### Kaggle GPU vs Ubuntu RTX3080 選擇：

**Kaggle 優勢：**
- 免費的 GPU 資源（每週約 30 小時）
- 無需本地設置，即開即用
- 預裝大部分深度學習套件
- 方便的筆記本分享和版本控制

**Kaggle 劣勢：**
- 檔案管理相對不便
- 終端操作受限
- 時間限制（每次最多 12 小時）
- 網路限制可能影響大模型下載

**Ubuntu RTX3080 優勢：**
- 完全控制權，無時間限制
- 更好的檔案管理和終端操作
- 可以進行長時間訓練
- 本地儲存，無需重複下載

**Ubuntu RTX3080 劣勢：**
- 需要自己設置環境
- 電費和硬體損耗
- 需要維護和更新

#### 具體建議：

1. **對於本課程的 SFT 訓練**：
   - 如果只是學習和實驗，Kaggle 完全夠用
   - RTX3080 10GB 對於 Qwen3-0.6B 模型是充足的
   - 可以兩者結合使用：Kaggle 做初步實驗，Ubuntu 做正式訓練

2. **選擇依據**：
   - **學習目的**：優先選擇 Kaggle
   - **生產目的**：建議使用 Ubuntu RTX3080
   - **大模型訓練**：Ubuntu RTX3080 更適合

3. **最佳實踐**：
   - 在 Kaggle 上快速原型和測試
   - 在本地 Ubuntu 上進行完整訓練
   - 使用 Git 同步代碼和配置

### 下一步學習建議

1. **深入理解 SFT 變體**：
   - 研究 LoRA、QLoRA 等參數效率微調方法
   - 了解指令工程和提示設計

2. **探索進階技術**：
   - 直接偏好優化（DPO）
   - 強化學習人類反饋（RLHF）
   - 多輪對話訓練

3. **實踐項目**：
   - 嘗試在不同領域的資料集上進行 SFT
   - 比較不同超參數設置的效果
   - 評估模型在各種任務上的表現

監督式微調是現代 AI 系統的核心技術之一，掌握它將為您在 AI 領域的發展打下堅實基礎！